In [2]:
import pandas as pd

opus = pd.read_csv("../data/opus_books.csv")
opus.head()

,en,es
0,"In the society of his nephew and niece, and th...","En compañía de su sobrino y sobrina, y de los ..."
1,"By a former marriage, Mr. Henry Dashwood had o...","De un matrimonio anterior, el señor Henry Dash..."
2,"By his own marriage, likewise, which happened ...","Además, su propio matrimonio, ocurrido poco de..."
3,"But the fortune, which had been so tardy in co...","Pero la fortuna, que había tardado tanto en ll..."
4,But Mrs. John Dashwood was a strong caricature...,Pero la señora de John Dashwood era una áspera...


In [3]:
import re
from collections import defaultdict

special_tokens = {
    "PAD": 0,
    "UNK": 1,
    "BOS": 2,
    "EOS": 3
}
vocab = special_tokens.copy()

def clean(text):
    # Use re to replace punctuation that is not a comma, question mark, or period with spaces
    text = re.sub(r'[^\w\s,?.!]',' ', text)
    text = text.strip()
    return text

def tokenize(text):
    # Split on consecutive whitespace and punctuation
    tokens = re.findall(r'\w+|[^\w\s]+|[\s]+', text)
    return tokens

opus_tokens = defaultdict(int)
for index, row in opus.iterrows():
    for key in ["en", "es"]:
        cleaned = clean(row[key])
        tokens = tokenize(cleaned)
        for token in tokens:
            opus_tokens[token] += 1

counter = 4
for index, token in enumerate(opus_tokens):
    # Filter out uncommon tokens
    # Add unknown token for rare words
    if opus_tokens[token] > 3:
        vocab[token] = counter
        counter += 1
    else:
        vocab[token] = 1 # Assign unknown id

In [4]:
import numpy as np

def encode(text, vocab):
    # Encode text as a list of integers
    tokens = tokenize(clean(text))
    encoded = np.array([vocab[token] for token in tokens])
    return encoded

reverse_vocab = {v: k for k, v in vocab.items()}
for k,v in special_tokens.items():
    reverse_vocab[v] = k

def decode(encoded, reverse_vocab):
    # Decode a list of integers into text
    decoded = "".join([reverse_vocab[token] for token in encoded])
    return decoded

In [15]:
data = []
for index, row in opus.iterrows():
    # Encode the English and Spanish sentences
    en_text = row["en"]
    es_text = row["es"]
    en = encode(en_text, vocab)
    es = encode(es_text, vocab)

    # Generate our prediction target
    target = np.roll(es, -1)
    target[-1] = 3 # EOS
    data.append({"en": en, "es": es, "en_text": en_text, "es_text": es_text, "target": target})

In [16]:
data[1000]

{'en': array([1415,    5, 4384,    5,  853,    5,   87,    5,    6,    5, 4385,
           5,    8,    5,  508,    5,    1,    5,   46,   12,    5,   10,
           5,    8,    5,  240,    5,    9,    5,   50,    5,    1,   21]),
 'es': array([4386,    5, 4387,    5,  102,    5,   35,    5, 4388,    5,   24,
           5,  715,    5,   28,    5, 4389,    5, 4390,    5,   96,    5,
         233,    5, 4391,    5,   26,    5,   24,    5,  155,    5,   25,
           5,   62,    5, 4392,   21]),
 'en_text': 'That crime has been the origin of every lesser one, and of all his present discontents."',
 'es_text': 'Ese crimen fue el origen de todos los males menores que le siguieron y de todo su actual descontento.',
 'target': array([   5, 4387,    5,  102,    5,   35,    5, 4388,    5,   24,    5,
         715,    5,   28,    5, 4389,    5, 4390,    5,   96,    5,  233,
           5, 4391,    5,   26,    5,   24,    5,  155,    5,   25,    5,
          62,    5, 4392,   21,    3])}

In [18]:
import math
class Embedding():
    def __init__(self, vocab_size, embed_dim):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        k = 1/math.sqrt(embed_dim)
        self.weights = np.random.rand(vocab_size, embed_dim) * 2 * k - k
        self.weights[0] = 0 # Zero out the padding embedding

    def forward(self, token_ids):
        # Cache for backward pass
        self.token_ids = token_ids
        # Return a matrix of embeddings
        # We could convert token_ids to a one_hot vector and multiply by the weights, but it is the same as selecting a single row of the matrix
        return self.weights[token_ids]

    def backward(self, grad, lr):
        for i, token_id in enumerate(self.token_ids):
            # Add the gradient to the embedding
            # We could convert the input to one-hot, and do input.T * grad, but it is the same as adding the gradient to the embedding
            # Subtracting the gradient is SGD optimization
            self.weights[token_id] -= grad[i] * lr

In [19]:
input_embed = Embedding(len(set(vocab.values())), 512)
input_embed.forward(data[0]["en"])

array([[ 0.02405603,  0.04253936, -0.01771518, ...,  0.04196606,
         0.01059922,  0.00290236],
       [ 0.00526091, -0.03544216,  0.0096714 , ...,  0.0370609 ,
         0.04180169, -0.03076386],
       [-0.03354411, -0.02107315, -0.03113892, ...,  0.01260971,
        -0.04156332, -0.01547263],
       ...,
       [ 0.00526091, -0.03544216,  0.0096714 , ...,  0.0370609 ,
         0.04180169, -0.03076386],
       [-0.03811562,  0.03132126, -0.01054451, ..., -0.00798627,
        -0.03331769,  0.0065835 ],
       [ 0.01550159,  0.00783828,  0.01947204, ...,  0.0398476 ,
        -0.0129053 , -0.03832633]])

array([[ 0.        ,  0.04253936,  0.        , ...,  0.04196606,
         0.01059922,  0.00290236],
       [ 0.00526091,  0.        ,  0.        , ...,  0.        ,
         0.04180169,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.01260971,
         0.        ,  0.        ],
       ...,
       [ 0.00526091,  0.        ,  0.0096714 , ...,  0.0370609 ,
         0.04180169, -0.03076386],
       [-0.03811562,  0.        , -0.01054451, ...,  0.        ,
         0.        ,  0.0065835 ],
       [ 0.        ,  0.00783828,  0.01947204, ...,  0.        ,
        -0.0129053 ,  0.        ]])

array([[ 0.92855015,  1.66255411, -0.73025494, ...,  1.63978755,
         0.39415737,  0.08850216],
       [ 0.23480195, -1.33519437,  0.40492317, ...,  1.46138918,
         1.64425061, -1.15474332],
       [-1.32633023, -0.83376943, -1.23133335, ...,  0.49658966,
        -1.64306226, -0.61256811],
       ...,
       [ 0.23480195, -1.33519437,  0.40492317, ...,  1.46138918,
         1.64425061, -1.15474332],
       [-1.49511875,  1.23420366, -0.41139434, ..., -0.31083881,
        -1.30652879,  0.26184852],
       [ 0.6034468 ,  0.31321756,  0.75381792, ...,  1.52549273,
        -0.47239533, -1.4351553 ]])